<a href="https://colab.research.google.com/github/Redcoder815/Deep_Learning_PyTorch/blob/main/MultilayerPerceptronClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

class MLP(nn.Module):
    def __init__(self, layer_sizes, activation=F.relu):
        super().__init__()
        self.activation = activation

        self.weights = nn.ParameterList()
        self.biases = nn.ParameterList()

        for in_dim, out_dim in zip(layer_sizes[:-1], layer_sizes[1:]):
            W = nn.Parameter(torch.randn(in_dim, out_dim) * 0.01)
            b = nn.Parameter(torch.zeros(out_dim))
            self.weights.append(W)
            self.biases.append(b)

    def forward(self, x):
        for i, (W, b) in enumerate(zip(self.weights, self.biases)):
            x = x @ W + b
            if i < len(self.weights) - 1:
                x = self.activation(x)
        return x  # logits

# --- Your MLP Class remains the same ---
# (Assumed to be defined above)

# 1. Generate Synthetic Data (Non-linear 'Moons' dataset)
X, y = make_moons(n_samples=1000, noise=0.1, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch Tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1) # Shape (N, 1)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# 2. Initialize Model, Optimizer, and Loss
model = MLP([2, 16, 16, 1]) # Added an extra layer for better capacity
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.BCEWithLogitsLoss()

# 3. Training Loop
epochs = 500
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    logits = model(X_train)
    loss = loss_fn(logits, y_train)

    # Backward pass
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        # Calculate accuracy
        model.eval()
        with torch.no_grad():
            # Apply sigmoid to get probabilities, then round to 0 or 1
            preds = torch.round(torch.sigmoid(model(X_test)))
            acc = (preds == y_test).float().mean()
        print(f"Epoch {epoch+1}/{epochs} | Loss: {loss.item():.4f} | Test Acc: {acc.item():.4f}")

# 4. Final Evaluation
model.eval()
with torch.no_grad():
    final_logits = model(X_test)
    final_preds = torch.round(torch.sigmoid(final_logits))
    final_acc = (final_preds == y_test).float().mean()
    print(f"\nFinal Test Accuracy: {final_acc.item() * 100:.2f}%")


Epoch 100/500 | Loss: 0.2528 | Test Acc: 0.8700
Epoch 200/500 | Loss: 0.2527 | Test Acc: 0.8700
Epoch 300/500 | Loss: 0.2527 | Test Acc: 0.8700
Epoch 400/500 | Loss: 0.2527 | Test Acc: 0.8700
Epoch 500/500 | Loss: 0.2527 | Test Acc: 0.8700

Final Test Accuracy: 87.00%


In [27]:

import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split

class MLP(nn.Module):
    def __init__(self, layer_sizes, activation=F.relu):
        super().__init__()
        self.activation = activation

        self.weights = nn.ParameterList()
        self.biases = nn.ParameterList()

        for in_dim, out_dim in zip(layer_sizes[:-1], layer_sizes[1:]):
            W = nn.Parameter(torch.randn(in_dim, out_dim) * 0.01)
            b = nn.Parameter(torch.zeros(out_dim))
            self.weights.append(W)
            self.biases.append(b)

    def forward(self, x):
        for i, (W, b) in enumerate(zip(self.weights, self.biases)):
            x = x @ W + b
            if i < len(self.weights) - 1:
                x = self.activation(x)
        return x  # logits

# 1. Generate Synthetic Multi-class Data (3 classes)
X, y = make_blobs(n_samples=1000, centers=3, n_features=2, cluster_std=1.0, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch Tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long) # CrossEntropyLoss expects Long/int labels
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

# 2. Initialize Model (2 features -> 16 hidden -> 3 classes)
model = MLP([2, 16, 3])
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

# 3. Training Loop
epochs = 200
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass: model returns raw logits (batch_size, 3)
    logits = model(X_train)
    loss = loss_fn(logits, y_train)

    # Backward pass
    loss.backward()
    optimizer.step()

    # Evaluate every 50 epochs
    if (epoch + 1) % 50 == 0:
        model.eval()
        with torch.no_grad():
            # Accuracy calculation using argmax to find the predicted class
            test_logits = model(X_test)
            preds = torch.argmax(test_logits, dim=1)
            acc = (preds == y_test).float().mean()
            print(f"Epoch {epoch+1:3d} | Loss: {loss.item():.4f} | Test Acc: {acc.item():.4f}")

# 4. Final Evaluation
model.eval()
with torch.no_grad():
    final_logits = model(X_test)
    final_preds = torch.argmax(final_logits, dim=1)
    final_acc = (final_preds == y_test).float().mean()
    print(f"\nFinal Test Accuracy: {final_acc.item() * 100:.2f}%")


Epoch  50 | Loss: 0.0009 | Test Acc: 1.0000
Epoch 100 | Loss: 0.0005 | Test Acc: 1.0000
Epoch 150 | Loss: 0.0004 | Test Acc: 1.0000
Epoch 200 | Loss: 0.0003 | Test Acc: 1.0000

Final Test Accuracy: 100.00%
